<a href="https://colab.research.google.com/github/Indresh0007/PySpark-Indresh/blob/main/UseCase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install Java and Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SQL to CSV using PySpark") \
    .getOrCreate()


In [4]:
!pip install sqlparse pandas


In [5]:
with open('/content/mysqlsampledatabase.sql', 'r', encoding='utf-8') as file:
    sql_content = file.read()


In [6]:
import re
import pandas as pd
from io import StringIO

# Regex to find all INSERT INTO statements
insert_statements = re.findall(r"INSERT INTO\s+(\w+)\s*\((.*?)\)\s*VALUES\s*(.*?);", sql_content, re.DOTALL)

table_data_dict = {}

for table_name, columns_str, values_blob in insert_statements:
    # Clean and split columns
    columns = [col.strip() for col in columns_str.split(',')]

    # Clean and split value rows
    raw_rows = re.findall(r"\((.*?)\)", values_blob, re.DOTALL)
    parsed_rows = []

    for row in raw_rows:
        # Split carefully respecting quoted strings
        items = list(csv.reader([row], skipinitialspace=True))[0]
        parsed_rows.append(items)

    # Create DataFrame
    df = pd.DataFrame(parsed_rows, columns=columns)
    table_data_dict[table_name] = df


In [7]:
import os

os.makedirs("csv_output", exist_ok=True)

for table, df in table_data_dict.items():
    output_path = f"csv_output/{table}.csv"
    df.to_csv(output_path, index=False)


In [8]:
!zip -r csv_files.zip csv_output
from google.colab import files
files.download("csv_files.zip")


  adding: csv_output/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
print(sql_content[:1000])  # Show first 1000 characters only


/* Create the database */
CREATE DATABASE  IF NOT EXISTS classicmodels;

/* Switch to the classicmodels database */
USE classicmodels;

/* Drop existing tables  */
DROP TABLE IF EXISTS productlines;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS offices;
DROP TABLE IF EXISTS employees;
DROP TABLE IF EXISTS customers; 
DROP TABLE IF EXISTS payments;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS orderdetails;

/* Create the tables */
CREATE TABLE productlines (
  productLine varchar(50),
  textDescription varchar(4000) DEFAULT NULL,
  htmlDescription mediumtext,
  image mediumblob,
  PRIMARY KEY (productLine)
);

CREATE TABLE products (
  productCode varchar(15),
  productName varchar(70) NOT NULL,
  productLine varchar(50) NOT NULL,
  productScale varchar(10) NOT NULL,
  productVendor varchar(50) NOT NULL,
  productDescription text NOT NULL,
  quantityInStock smallint(6) NOT NULL,
  buyPrice decimal(10,2) NOT NULL,
  MSRP decimal(10,2) NOT NULL,
  PRIMARY KEY (productCode),
  F

In [10]:
!pip install sqlparse pandas


In [11]:
import sqlparse
import re
import pandas as pd
import csv
import os
from io import StringIO


In [12]:
with open('/content/mysqlsampledatabase.sql', 'r', encoding='utf-8') as file:
    sql_content = file.read()


In [13]:
# Clean up SQL content
statements = sqlparse.split(sql_content)

table_data_dict = {}

for statement in statements:
    if statement.strip().upper().startswith("INSERT INTO"):
        # Extract table name and values using regex
        match = re.match(r"INSERT INTO\s+(\w+)\s*\((.*?)\)\s*VALUES\s*(.*)", statement, re.DOTALL | re.IGNORECASE)
        if match:
            table_name, columns_str, values_blob = match.groups()
            columns = [col.strip() for col in columns_str.split(',')]

            # Split values while preserving quotes
            raw_rows = re.findall(r"\(([^()]+?)\)", values_blob)
            parsed_rows = []

            for row in raw_rows:
                reader = csv.reader([row], skipinitialspace=True)
                parsed_rows.append(next(reader))

            # Create and store the DataFrame
            df = pd.DataFrame(parsed_rows, columns=columns)
            table_data_dict[table_name] = df


In [14]:
os.makedirs("csv_output", exist_ok=True)

for table, df in table_data_dict.items():
    df.to_csv(f"csv_output/{table}.csv", index=False)


In [15]:
!zip -r csv_files.zip csv_output
from google.colab import files
files.download("csv_files.zip")


updating: csv_output/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
from google.colab import files
uploaded = files.upload()

Saving mysqlsampledatabase.sql to mysqlsampledatabase (1).sql


In [17]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar xf spark-3.3.2-bin-hadoop3.tgz
!pip install -q findspark

import os, findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"
findspark.init()

In [18]:
import os
os.listdir()  # check if file is here


['.config',
 'spark-3.3.2-bin-hadoop3',
 'spark-3.4.1-bin-hadoop3.tgz',
 'spark-3.3.2-bin-hadoop3.tgz',
 'csv_output',
 'mysqlsampledatabase (1).sql',
 'csv_files.zip',
 'mysqlsampledatabase.sql',
 'spark-3.4.1-bin-hadoop3',
 'sample_data']

In [21]:
# Clean MySQL-specific syntax for SQLite compatibility
import re

# Read the SQL content from the uploaded file
with open('/content/mysqlsampledatabase (1).sql', 'r', encoding='utf-8') as file:
    sql_script = file.read()

sql_script = sql_script.replace("CREATE DATABASE  IF NOT EXISTS classicmodels;", "")
sql_script = sql_script.replace("USE classicmodels;", "")

# Replace MySQL data types with SQLite-friendly ones
sql_script = re.sub(r"mediumtext", "TEXT", sql_script, flags=re.IGNORECASE);
sql_script = re.sub(r"mediumblob", "BLOB", sql_script, flags=re.IGNORECASE);
sql_script = re.sub(r"smallint\(\d+\)", "INTEGER", sql_script, flags=re.IGNORECASE);
sql_script = re.sub(r"decimal\(\d+,\d+\)", "REAL", sql_script, flags=re.IGNORECASE);
sql_script = re.sub(r"int\(\d+\)", "INTEGER", sql_script, flags=re.IGNORECASE);

# Remove AUTO_INCREMENT and ENGINE definitions
sql_script = re.sub(r"AUTO_INCREMENT", "", sql_script, flags=re.IGNORECASE);
sql_script = re.sub(r"ENGINE\s*=\s*\w+\s*DEFAULT CHARSET=\w+", "", sql_script, flags=re.IGNORECASE);

# Fix escaped single quotes
sql_script = sql_script.replace("\\'", "''")

In [23]:
import sqlite3

conn = sqlite3.connect("sample.db")
conn.executescript(sql_script)

In [24]:
import os
if os.path.exists("classicmodels.db"):
    os.remove("classicmodels.db")

import sqlite3

conn = sqlite3.connect("classicmodels.db")
conn.executescript(sql_script)
conn.commit()


In [25]:
import pandas as pd

# Get table names
tables = [t[0] for t in conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()]

# Export each table to CSV
for table_name in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    df.to_csv(f"{table_name}.csv", index=False)
    print(f"{table_name}.csv exported")


productlines.csv exported
products.csv exported
offices.csv exported
employees.csv exported
customers.csv exported
payments.csv exported
orders.csv exported
orderdetails.csv exported


In [26]:
from google.colab import files

for table_name in tables:
    files.download(f"{table_name}.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
!ls

 classicmodels.db	        offices.csv	   sample.db
 csv_files.zip		        orderdetails.csv   spark-3.3.2-bin-hadoop3
 csv_output		        orders.csv	   spark-3.3.2-bin-hadoop3.tgz
 customers.csv		        payments.csv	   spark-3.4.1-bin-hadoop3
 employees.csv		        productlines.csv   spark-3.4.1-bin-hadoop3.tgz
'mysqlsampledatabase (1).sql'   products.csv
 mysqlsampledatabase.sql        sample_data


In [28]:
import pandas as pd
from google.colab import files

# Get table names
tables = [t[0] for t in conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()]

# Export each table to CSV and JSON
for table_name in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

    # Save as CSV
    csv_file = f"{table_name}.csv"
    df.to_csv(csv_file, index=False)

    # Save as JSON (records format is easier for APIs)
    json_file = f"{table_name}.json"
    df.to_json(json_file, orient="records", indent=2)

    print(f"{table_name}: CSV and JSON exported")

# Download all files
for table_name in tables:
    files.download(f"{table_name}.csv")
    files.download(f"{table_name}.json")


productlines: CSV and JSON exported
products: CSV and JSON exported
offices: CSV and JSON exported
employees: CSV and JSON exported
customers: CSV and JSON exported
payments: CSV and JSON exported
orders: CSV and JSON exported
orderdetails: CSV and JSON exported


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
import pandas as pd
import zipfile
from google.colab import files

# Get table names
tables = [t[0] for t in conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()]

# Export CSV and JSON
for table_name in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

    # Save CSV
    df.to_csv(f"{table_name}.csv", index=False)

    # Save JSON
    df.to_json(f"{table_name}.json", orient="records", indent=2)

# Create ZIP
zip_filename = "classicmodels_data.zip"
with zipfile.ZipFile(zip_filename, 'w') as zf:
    for table_name in tables:
        zf.write(f"{table_name}.csv")
        zf.write(f"{table_name}.json")

# Download ZIP
files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>